### Equipe:

Arthur Wanderley Ferreira dos Santos - awfs@cesar.school

Thiago Wanderley Amorim - twa@cesar.school

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00


In [4]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

### Dividindo Documentos - Splitting/Chunking

In [5]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [6]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [8]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.9/411.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29


In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.1 MB/s eta 0:00:00


In [11]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [13]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [15]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [18]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, há plano de saúde como benefício no CESAR. Além disso, também são oferecidos plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [22]:
import requests
import fitz  # PyMuPDF

In [23]:
# Função para baixar o PDF do Google Drive
def download_pdf_from_google_drive(drive_file_id, output_path):
    url = f"https://drive.google.com/uc?id={drive_file_id}&export=download"
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(output_path, "wb") as pdf_file:
            for chunk in response.iter_content(chunk_size=1024):
                pdf_file.write(chunk)
        print(f"PDF baixado com sucesso em: {output_path}")
    else:
        print(f"Erro ao baixar o PDF. Código de status: {response.status_code}")

In [25]:
# Função para extrair texto do PDF
def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page_num, page in enumerate(pdf, start=1):
            text += page.get_text()
    return text

In [26]:
# Configurar o ID do arquivo do Google Drive e caminho de saída
drive_file_id = "1i56bnFd_YYIYaGvDjawHz5383pdM2egj"  # Substitua pelo ID real do arquivo
output_pdf_path = "Revista_PARPEL_2024.pdf"

# Baixar o arquivo PDF
download_pdf_from_google_drive(drive_file_id, output_pdf_path)

# Extrair texto do PDF
pdf_text = extract_text_from_pdf(output_pdf_path)
print("Texto extraído do PDF com sucesso!")

# Exibir uma parte do texto para validação
print(pdf_text[:500])  # Mostra os primeiros 500 caracteres do texto extraído

PDF baixado com sucesso em: Revista_PARPEL_2024.pdf
Texto extraído do PDF com sucesso!
1
PA R / P E L  2 0 2 4  -  P L A N O  D A  O P E R A Ç Ã O  E L É T R I C A  D E  M É D I O  P R A Z O  D O  S I N
2
PA R / P E L  2 0 2 4  -  P L A N O  D A  O P E R A Ç Ã O  E L É T R I C A  D E  M É D I O  P R A Z O  D O  S I N
Alexandre Nunes Zucarato
Diretor de Planejamento - ONS
Mensagem do Diretor
O Operador Nacional do Sistema Elétrico 
elabora anualmente o Plano da Operação 
Elétrica de Médio Prazo do SIN (PAR/PEL) 
com as avaliações do desempenho elétrico 
do Sistema Interligado Nacio


In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(text, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text)
    return chunks

# Dividir o texto extraído
chunks = split_text(pdf_text)

In [28]:
from langchain.embeddings import OpenAIEmbeddings

# Gerar embeddings para os chunks
embeddings = OpenAIEmbeddings()
chunk_embeddings = [embeddings.embed_query(chunk) for chunk in chunks]

<ipython-input-28-1b6b631cb32b>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [29]:
from langchain.vectorstores import FAISS

# Criar o índice FAISS
vectorstore = FAISS.from_texts(chunks, embeddings)

# Salvar o índice localmente (opcional)
vectorstore.save_local("faiss_index")

In [36]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

# Configurar o modelo LLM
llm = ChatOpenAI(model="gpt-4")

# Prompt para combinar documentos (opcionalmente ajustável)
qa_prompt = PromptTemplate.from_template(
    "Use as informações a seguir para responder à pergunta com precisão.\n\n{context}\n\nPergunta: {question}\nResposta:"
)

# Criar um chain para combinar os documentos
combine_documents_chain = load_qa_chain(llm, chain_type="stuff", prompt=qa_prompt)

# Configurar o retriever do vetor
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Criar a RetrievalQA chain
qa_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=combine_documents_chain
)



In [37]:
# Fazer uma consulta
query = "Qual é o tema principal do documento?"
response = qa_chain.run(query)

print(response)

O tema principal do documento é o planejamento e operação do Sistema Interligado Nacional (SIN), com foco em questões como a ampliação e reforço das instalações, ajuste cronológico do plano de expansão da transmissão, impactos da geração distribuída na segurança elétrica, e desafios associados à conexão de grandes cargas. O documento também aborda a necessidade de um novo modelo de coordenação entre o Operador Nacional do Sistema Elétrico (ONS) e os agentes operadores dos recursos distribuídos.


In [38]:
# Fazer uma consulta
query = "Quais os números das principais infomrações mostrados no documento?"
response = qa_chain.run(query)

print(response)

Os números principais mostrados no documento são: 

- Impactos da Geração Distribuída na Segurança Elétrica (06)
- Dificuldades Técnicas e Regulatórias da Rede DIT (07)
- Desafios Associados à Conexão de Grandes Cargas ou Grandes Consumidores – Hidrogênio Verde (H2V) e Data Centers (08)
- Metodologia para Subsidiar a Indicação de Compensadores Síncronos em Pontos Estratégicos do SIN (09)
- A capacidade instalada de Micro e Minigeração Distribuída (MMGD) ultrapassa a marca de 33 GW.
